In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import numpy as np
import argparse
import glob
import os

import matplotlib.pyplot as plt

class MLP(nn.Module):
    def __init__(self, n_units1, n_units2):
        super(MLP, self).__init__()
        self.l1 = nn.Conv2d(3, n_units1, 5, stride=1, padding=2)
        self.l3 = nn.Conv2d(n_units1, 1, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l3(x)
        return F.sigmoid(x)

def train(args, model, device, dataloader, optimizer, epoch):
    model.train()
    lossfun = nn.BCELoss()
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = lossfun(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.item()))
            if args.dry_run:
                break

class WhitelineDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        imw = 320
        imh = 240
        x = np.zeros((0, 3, imh, imw), dtype=np.float32)
        t = np.zeros((0, 1, imh, imw), dtype=np.float32)
        jpgfiles = glob.glob('*.jpg')
        print(jpgfiles)
        for f in jpgfiles:
            plt.figure()
            img = Image.open(f).resize((imw, imh))
            plt.imshow(np.array(img))
            a = np.asarray(img).transpose(2,0,1).astype(np.float32)/255.
            a1 = np.expand_dims(a,axis=0)
            x = np.append(x, a1, axis=0)

            lfile = os.path.splitext(f)[0] + '_label.png'
            print(lfile)
            img = Image.open(lfile).resize((imw, imh))
            img = img.convert('L')
            timg = Image.fromarray(np.asarray(img))
            #plt.figure()
            #plt.imshow(timg)
            a = np.asarray(img).astype(np.float32) > 0.01
            a = a.astype(np.float32)
            print(np.sum(a))
            a1 = np.expand_dims(a,axis=0)
            a1 = np.expand_dims(a1,axis=0)
            t = np.append(t, a1, axis=0)
            #timg = Image.fromarray(a * 255, 'F')
            #plt.imshow(timg)
        self.data  = x
        self.label = t

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #print(self.data.shape)
        #print(self.label.shape)
        images =  self.data[idx, :, :, :]
        labels = self.label[idx, :, :]

        return (images, labels)


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

parser = argparse.ArgumentParser()
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=14, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                    help='learning rate (default: 1.0)')
parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                    help='Learning rate step gamma (default: 0.7)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--dry-run', action='store_true', default=False,
                    help='quickly check a single pass')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
args = parser.parse_args(args=[])
dataset = WhitelineDataset('./')
train_loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
model = MLP(4, 2).to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
for i in range(2000):
    train(args, model=model, device=device, dataloader=train_loader, optimizer=optimizer, epoch=i)

In [ ]:
cpu = torch.device("cpu")
def eval_image(fname, thre):
    imw = 320
    imh = 240
    model.to(cpu)
    testx = np.zeros((0, 3, imh, imw), dtype=np.float32)

    #y2 = (model.y.data)
    #plt.imshow(y2[0][0])
    img = Image.open(fname).convert('RGB').resize((imw,imh))
    a = np.asarray(img).transpose(2,0,1).astype(np.float32)/255.
    a1 = np.expand_dims(a,axis=0)
    #print(a1.shape)
    testx = np.append(testx, a1, axis=0)

    import time
    t0 = time.time()
    testy = model(torch.FloatTensor(testx))
    print('forward time [s]: ' + str(time.time()-t0))

    #fig, axs = plt.subplots(1,3)
    #plt.imshow(img, ax=axs[0])
    imd = Image.new('RGB', (imw*2, imh))
    imd.paste(img)
    thimg = (testy.detach().to(cpu).numpy()[0][0] > thre)
    print(thimg.shape)
    print(f'max {np.max(thimg)}')
    print(f'min {np.min(thimg)}')
    thimg = thimg.astype(np.uint8) * 255

    thimg = Image.fromarray(thimg)
    plt.imshow(thimg)

    imd.paste(thimg, (imw, 0))
    plt.imshow(imd)

In [ ]:
eval_image('000050.png', 0.5)

In [ ]:
eval_image('000003.jpg', 0.3)

In [ ]:
eval_image('000013.jpg', 0.2)

In [ ]:
eval_image('000047.jpg', 0.8)

In [ ]:
eval_image('000034.jpg', .5)